In [9]:
#import our libraries

import csv
import numpy as np
from numpy import genfromtxt
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.activations import relu, linear
from tensorflow.keras.layers import Dense
import tabulate

In [10]:
#define our xtrain our ytrain 

item_train = genfromtxt('data/content_user_train.csv',delimiter=',')
user_train = genfromtxt('data/content_user_train.csv', delimiter=',')
y_train = genfromtxt('data/content_y_train.csv',delimiter=',')


In [11]:
#Deleting unimportant datas from our database

item_input_num = item_train.shape[1]-3
user_input_num = user_train.shape[1]-3

item_train= item_train[:,3:]
user_train=user_train[:,3:]

user_input_num

14

In [12]:

#Scaling our data to make the model work faster 

ScalerItem1 = StandardScaler()

ScalerItem1.fit(item_train)
item_train =ScalerItem1.transform(item_train)

ScalerItem2 = StandardScaler()
ScalerItem2.fit(user_train)
user_train = ScalerItem2.transform(user_train)


ScalerTarget = MinMaxScaler((-1,1))
ScalerTarget.fit(y_train.reshape(-1,1))
y_train=ScalerTarget.transform(y_train.reshape(-1,1))

In [13]:
user_train , user_test = train_test_split(user_train,train_size=0.8, shuffle=True,random_state=1)
item_train , item_test = train_test_split(item_train,train_size=0.8, shuffle = True , random_state=1)
y_train , y_test =train_test_split(y_train,train_size=0.8,shuffle=True , random_state=1)

In [14]:



tf.random.set_seed(1)

item_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32),
])


user_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32),
])


input_user = tf.keras.layers.Input(shape=(user_input_num,))
input_item = tf.keras.layers.Input(shape=(item_input_num,))


vu = user_model(input_user)
vi = item_model(input_item)



output = tf.keras.layers.Dot(axes=1)([vi, vu])


model = tf.keras.Model([input_item, input_user], output)


model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 14)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, 14)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_2        │ (None, 32)        │     40,864 │ input_layer_5[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_3        │ (None, 32)        │     40,864 │ input_layer_4[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_1 (Dot)         │ (None, 1)         │          0 │ sequential_2[0][… │
│                     │                   │            │ sequential_3[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 81,728 (319.25 KB)

 Trainable params: 81,728 (319.25 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
tf.random.set_seed(1)
cost_fn=tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer=opt,loss=cost_fn)

In [ ]:
tf.random.set_seed(1)

model.fit([user_train,item_train],y_train,epochs=30)

Epoch 1/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.6544
Epoch 2/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1425
Epoch 3/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1413
Epoch 4/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1399
Epoch 5/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1397
Epoch 6/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1398
Epoch 7/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1398
Epoch 8/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1393
Epoch 9/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1402
Epoch 10/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1401
Epoch 11/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1381
Epoch 12/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1379
Epoch 13/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1380
Epoch 14/30
1273/1273 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1375
Epoch 15/30
1273/1273 ━━━━━━━

In [ ]:
model.evaluate([user_test,item_test],y_test)

319/319 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1409


0.1370810866355896

In [ ]:

movie_list = genfromtxt('data/content_item_vecs.csv',delimiter=',')

reshaped_movie_list=movie_list[:,3:]



#lets make a user who watched shrek, spy kids ,final fantasy and harry potter
user = np.zeros(reshaped_movie_list.shape)
user[10]=[5,0,0,0,0,5,0,0,0,0,5,0,0,5]
user[18]=[0,5,5,5,5,0,0,0,5,0,0,5,0,0]
user[31]=[0,5,5,0,0,0,0,0,5,0,0,0,5,0]
user[57]=[0,5,0,5,0,0,0,0,5,0,0,0,0,0]

user =ScalerItem2.transform(user)
reshaped_movie_list= ScalerItem1.transform(reshaped_movie_list)
#we will recommend him 5
recommendation =[]
recommendation_name=[]

maximum=0

for i in range(0,user.shape[0]):
    
  
    yhat =model.predict([user[i].reshape(1,-1),reshaped_movie_list[i].reshape(1,-1)])
    yhat = ScalerTarget.inverse_transform(yhat)
    recommendation.append(yhat)
    if yhat>maximum:
        movie=i
        maximum=yhat
        
    recommendation_name.append(i+1)

  


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━

In [ ]:
print(movie)


345


array([[2.977323]], dtype=float32)